In [1]:
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf

if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")
else:
    print("The installed version of TensorFlow does not include GPU support.")
    
tf.logging.set_verbosity(0)
import pandas as pd
import matplotlib 
%matplotlib inline

plt = matplotlib.pyplot
from gpflow.likelihoods import Gaussian
from ordinal import Ordinal
from gpflow.kernels import RBF, White
from gpflow.models.gpr import GPR
from gpflow.models.svgp import SVGP
from gpflow.training import AdamOptimizer, ScipyOptimizer
import gpflow.training.monitor as mon
from gpflow.kernels import Matern32
from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp
from doubly_stochastic_dgp.dgp import DGP
import gpflow
matplotlib.rcParams['figure.figsize'] = (12, 6)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12918200221195333655
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7905922253
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2060363817790192070
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
The installed version of TensorFlow includes GPU support.


**Model**

In [2]:
def make_dgp_models(X, Y, Z,likelihood,layers):
    models, names = [], []
    for L in layers:
        D = X.shape[1]
        N = X.shape[0]
        print('Design Matrix Shape:'+str(D)+'X'+str(N))
        
        # the layer shapes are defined by the kernel dims, so here all hidden layers are D dimensional 
        kernels = []
        for l in range(L):
            kernels.append(RBF(D,variance = 1.5,lengthscales=0.5,ARD=True))

        # between layer noise (doesn't actually make much difference but we include it anyway)
        for kernel in kernels[:-1]:
            white = White(D, variance=1e-5)
#             tf.is_variable_initialized(white.variance).mark_used()
            kernel += white

        mb = 1000 if X.shape[0] > 1000 else None 
        model = DGP(X, Y, Z, kernels, likelihood, num_samples=10, minibatch_size=mb)

        # start the inner layers almost deterministically 
        for layer in model.layers[:-1]:
            layer.q_sqrt = layer.q_sqrt.value * 1e-5

        models.append(model)
        names.append('DGP{} {}'.format(L, len(Z)))
        model.compile()
    
    return models, names

def make_single_layer_model(X,Y,Z,likelihood):
    D = X.shape[1]
    m_svgp = SVGP(X, Y, RBF(D,variance = 1.5,lengthscales=0.5,ARD=True), likelihood, Z.copy())
    name = 'SVGP'+str(len(Z))
    return m_svgp, name



**Prediction**<br>
We'll calculate test absolute error in batches (so the larger datasets don't cause memory problems)

For the DGP models we need to take an average over the samples for the absolute error. The predict_density function already does this internally

In [3]:
def batch_assess(model, assess_model, X, Y):
    n_batches = max(int(X.shape[0]/1000.), 1)
    AbsErr = []
    for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
        abserr = assess_model(model, X_batch, Y_batch)
        AbsErr.append(abserr)
        
    MAErr = np.average(np.concatenate(AbsErr, 0))
    
    return MAErr

def assess_single_layer(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch)
    mean = np.average(m, 0)
    abserr = np.absolute((mean - Y_batch))
    return abserr
S = 100
def assess_sampled(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch, S)
    
    mean = np.average(m, 0)
    abserr = np.absolute((mean - Y_batch))
    return abserr

In [4]:
iterations_few = 100
iterations_many = 5000
s = '{:<16}  Mean Absolute Error : {:.4f} +- {:.4f}'
iterations = 20000


class CustomTestPerformanceTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, file_writer, model, Xt, Yt,S = 100):
        super().__init__(file_writer, model)
        self.Xt = Xt
        self.Yt = Yt
        self.Y_std = Y_std
        self._full_test_err = tf.placeholder(gpflow.settings.float_type, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.float_type, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar(model.name +"/test_rmse", self._full_test_err),
                                         tf.summary.scalar(model.name +"/test_nlpp", self._full_test_nlpp)])
        self.S = S
    
    def batch_assess(self,assess_model, X, Y,minibatch_size = 1000.):
        n_batches = max(int(X.shape[0]/minibatch_size), 1)
        AbsErr = []
        for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
            abserr = assess_model(X_batch, Y_batch)
            AbsErr.append(abserr)

        MAErr = np.average(np.concatenate(AbsErr, 0))

        return MAErr
      

    def assess_sampled(self, X_batch, Y_batch):
        m, v = self.model.predict_y(X_batch, self.S)

        mean = np.average(m, 0)
        abserr = np.absolute((mean - Y_batch))
        return abserr
    
       
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:        
        test_mean_abs_err = self.batch_assess(self.assess_sampled, self.Xt, self.Yt)            
        self._eval_summary(context, {self._full_test__mean_abs_err: test_mean_abs_err})

        





In [5]:
class CustomPrintTask(mon.MonitorTask):
    def __init__(self,  model): 
        super().__init__()
        self.model = model
        
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:        
        print(self.model.likelihood.likelihood.bin_start)
        print(self.model.likelihood.likelihood.bin_width)

In [6]:
testString =  "./abalone_dataset/10bins/abalone_test_10."
trainString = "./abalone_dataset/10bins/abalone_train_10."

MAErr=[]

for j in range(1,21):
    flag = True
    dfTest = pd.read_csv(testString + str(j),header = None,sep = ' ')
    dfTrain = pd.read_csv(trainString +  str(j),header = None,sep = ' ')
    train= np.array(dfTrain)
    Y = train[:,-1:]
    X = train[:, :-1]
    test  = np.array(dfTest)
    YT = test[:,-1:]
    XT = test[:, :-1]
    YT = YT -1
    Y = Y -1
#     bin_edges = []
#     uniqueY = np.unique(Y)
#     for i in range(uniqueY.size - 1 ):
#         bin_edges.append((uniqueY[i] + uniqueY[i+1])/2)
#     bin_edges = np.array(bin_edges)    
    
    # construct ordinal likelihood - bin_edges is the same as unique(Y) but centered
#     bin_edges = np.array(np.arange(np.unique(Y).size - 1), dtype=float)
#     bin_edges = bin_edges - bin_edges.mean()
    
    # bin_edges = bin_edges - bin_edges.mean()
    likelihood= Ordinal(bin_start=0, bin_width = 2, num_edges = np.unique(Y).size - 1)
    Z = kmeans2(X, 100, minit='points')[0]
    models_dgp, names_dgp = make_dgp_models(X, Y, Z,likelihood,[2])
    maerr=[]
    for m, name in zip(models_dgp, names_dgp):
        
#         print_task = CustomPrintTask(m).with_name('print_tboard')\
#             .with_condition(mon.PeriodicIterationCondition(1))\
#             .with_exit_condition(True)
#         monitor_tasks = [print_task]
#         session = m.enquire_session()
#         global_step = mon.create_global_step(session)
        
#         opti miser = AdamOptimizer(0.01)
        
#         with mon.Monitor(monitor_tasks, session,global_step, print_summary=True) as monitor:
#             optimiser.minimize(m, step_callback=monitor, maxiter=iterations, global_step=global_step)
       
        AdamOptimizer(0.01).minimize(m, maxiter=iterations)
        maerr.append(batch_assess(m, assess_sampled, XT, YT))
    model_svgp, svgp_name = make_single_layer_model(X,Y,Z,likelihood)
    ScipyOptimizer().minimize(model_svgp, maxiter=iterations)
    maerr.append(batch_assess(model_svgp, assess_single_layer, XT, YT))
    
    MAErr.append(maerr)

MAErr=np.array(MAErr)    


  
# tempInc = 2
# temp1 = 320000 iterations

# temp2 = 0
# i =0 
# #     while tempInc > .00001:
# #         if(flag):
# AdamOptimizer(0.01).minimize(m,maxiter=4000)
# #         flag = false
# #         AdamOptimizer(0.01).minimize(m,maxiter=500)
# #     ScipyOptimizer().minimize(m,maxiter=4000)
# Xs = XT
# S = 10
# mean,var= m.predict_y(Xs,S)
# fmean = np.average(mean,0)
# fvar = np.average(var,0)
# #     temp2 = np.average(np.absolute((np.round(fmean) - YT)))
# #     tempInc = temp1 - temp2
# #     temp1 = temp2
# print(testString + str(j))
# print(np.average(np.absolute((np.round(fmean) - YT))))
# #     i = i +1 
# #     print(i )


Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10
Design Matrix Shape:10X1000
10 10


In [7]:
meanMAErr = np.mean(MAErr,0)
stdMAErr = np.std(MAErr,0)

for i in range(len(names_dgp)):
    print(s.format(names_dgp[i],meanMAErr[i],stdMAErr[i]))
    
print(s.format(svgp_name,meanMAErr[-1],stdMAErr[-1]))

DGP2 100          Mean Absolute Error : 0.5761 +- 0.0087
SVGP100           Mean Absolute Error : 0.5614 +- 0.0076


In [ ]:
var 1.5 l 0.5
DGP2 100          Mean Absolute Error : 0.6173 +- 0.0067
SVGP100           Mean Absolute Error : 0.5735 +- 0.0036

In [ ]:
var 0.5 l 0.5
DGP2 100          Mean Absolute Error : 0.6338 +- 0.0097
SVGP100           Mean Absolute Error : 0.5694 +- 0.0009

In [8]:
var 2 l 0.5
DGP2 100          Mean Absolute Error : 0.6162 +- 0.0019
SVGP100           Mean Absolute Error : 0.5743 +- 0.0037

SyntaxError: invalid syntax (<ipython-input-8-2e078b4bb02f>, line 1)

In [ ]:
var 2 l 2
DGP2 100          Mean Absolute Error : 1.6667 +- 0.0022
SVGP100           Mean Absolute Error : 0.5716 +- 0.0002

var 2 l= 1.0
DGP2 100          Mean Absolute Error : 0.6573 +- 0.0005
SVGP100           Mean Absolute Error : 0.5727 +- 0.0004

In [ ]:
models_dgp[0].likelihood.likelihood.bin_start

In [ ]:
models_dgp[0].likelihood.likelihood.bin_width

20000 iterations
DGP2 100          Mean Absolute Error : 0.5743 +- 0.0088
SVGP100           Mean Absolute Error : 0.5634 +- 0.0075